In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns=999
pd.options.display.max_rows=999
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
#import graphviz
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix

# %%
## IMPORT PYTHON LIBRARIES ##
import time
import glob
import dill
import joblib
import warnings
import pandas as pd
import numpy as np
import xgboost as xgb
import lime.lime_tabular
from datetime import date
from sklearn import tree
from sklearn import metrics
from collections import defaultdict
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

import seaborn as sns
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from matplotlib.patches import Patch

## KONFIGURASI PANDAS DAN PLOT ##
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.option_context("display.max_rows", 100)
rcParams['figure.figsize'] = 12, 4

In [12]:
df_bill = pd.read_csv('probill.csv')
#df_trx = pd.read_csv('20210625_dataset_per_nasabah.csv')
df_mar =pd.read_csv('data_trx_ceria_march_filtered.csv')
df_fraud = pd.read_csv('Referensi_ODA_Fraud.csv')
#df_label = pd.read_csv('20210628_dataset_per_nasabah_ready_for_cleaning.csv')
df_label_maret = pd.read_csv('bills_maret.csv')

In [13]:
df_bill.head()

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts
0,9910107,1.895121e+15,1.079010e+14,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-04-08 17:00:00,0.0,0.0,433295.0,433295.0,0.0,3.373333e+02,414.00,0.00,414.0,NaN,27.0,NaN,11178.0,-11178.0,20210324
1,9910111,1.895121e+15,1.750104e+13,2021-04-08 17:00:00,2021-04-24 17:00:00,2021-04-24 17:00:00,34754.0,34754.0,1994057.0,1994057.0,0.0,3.518692e+05,4041736.05,40857.05,56989.0,112.0,1064.0,4.543338e+08,449927408.0,4406416.0,20210424
2,9910124,1.895121e+15,5.901075e+12,2020-07-08 17:00:00,2020-07-24 17:00:00,2020-07-19 17:00:00,82721.0,82721.0,2243124.0,2243124.0,0.0,1.122287e+07,14448528.75,3089354.63,12097240.0,180.0,1530.0,5.994012e+08,240308130.0,359093103.6,20200724
3,9910153,1.895121e+15,2.430106e+13,2021-04-08 17:00:00,2021-04-24 17:00:00,2021-04-22 17:00:00,1186966.0,1186966.0,17182495.0,17182495.0,0.0,6.112633e+05,6127227.83,50228.83,55229.0,196.0,1316.0,1.013991e+09,853139084.0,160852377.0,20210424
4,991030,1.895121e+15,6.863010e+14,2020-10-08 17:00:00,2020-10-24 17:00:00,2020-10-19 17:00:00,588784.0,588784.0,3370794.0,3370794.0,0.0,8.978032e+06,15538041.00,5978808.00,7909142.0,1102.0,2291.0,8.095693e+08,854426246.0,-44856968.0,20201024


In [14]:
df_label_maret

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd,collectibility,hold_flag,hold,limit,oda_status,posisi
0,99143153,1895121000014127,16501031342508,2021-03-23 17:00:00,2021-04-08 17:00:00,2021-04-03 17:00:00,1020232.0,1020232.0,1020232.0,1020232.0,0.0,2.272930e+07,25021783.34,20598519.99,22812000,182.0,936.0,2.048392e+08,180741990.0,2.409723e+07,20210408,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210408.0
1,99143153,1895121000014127,16501031342508,2021-04-23 17:00:00,2021-05-09 17:00:00,2021-05-04 17:00:00,1287272.0,1287272.0,2307504.0,2307504.0,0.0,2.100589e+07,22046691.34,18675339.34,21022370,270.0,972.0,2.601165e+08,320305941.0,-6.018941e+07,20210509,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210509.0
2,99143153,1895121000014127,16501031342508,2021-05-23 17:00:00,2021-06-08 17:00:00,2021-06-03 17:00:00,1287272.0,1287272.0,3594776.0,3594776.0,0.0,1.373584e+07,20029453.99,10478869.99,12293953,84.0,868.0,5.606709e+07,345155440.0,-2.890884e+08,20210608,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210608.0
3,99143154,1895121000014128,13901129110503,2021-03-15 17:00:00,2021-03-31 17:00:00,2021-04-15 17:00:00,0.0,0.0,0.0,0.0,0.0,4.608916e+05,10349926.60,49001.60,75528,112.0,420.0,6.753460e+07,361420500.0,-2.938859e+08,20210331,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210331.0
4,99143154,1895121000014128,13901129110503,2021-04-15 17:00:00,2021-05-01 17:00:00,2021-05-15 17:00:00,0.0,0.0,0.0,0.0,0.0,5.036160e+04,58501.60,48501.60,48502,25.0,75.0,7.500000e+06,7387500.0,1.125000e+05,20210501,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210501.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36854,99199056,1895121000028732,79901042281536,2021-05-08 17:00:00,2021-05-24 17:00:00,2021-06-08 17:00:00,2032670.0,0.0,3163296.0,0.0,3163296.0,6.663077e+05,1233641.00,83641.00,683641,108.0,81.0,4.603500e+07,16416000.0,2.961900e+07,20210524,2021-03-30 17:00:00,45.0,2.0,N,0.0,6250000.0,Active,20210524.0
36855,99199056,1895121000028732,79901042281536,2021-06-08 17:00:00,2021-06-24 17:00:00,2021-06-08 17:00:00,2961776.0,0.0,6125072.0,0.0,6125072.0,8.938804e+04,95626.00,87626.00,87626,27.0,135.0,2.295000e+03,29729700.0,-2.972740e+07,20210622,2021-03-30 17:00:00,74.0,2.0,N,0.0,6250000.0,Active,20210622.0
36856,99199057,1895121000028733,26101019958503,2021-04-08 17:00:00,2021-04-24 17:00:00,2021-05-01 17:00:00,221300.0,221300.0,221300.0,221300.0,0.0,1.299460e+05,1566946.00,16946.00,20946,56.0,196.0,7.700000e+07,75894000.0,1.106000e+06,20210424,2021-03-30 17:00:00,15.0,1.0,Y,221300.0,2500000.0,Active,20210424.0
36857,99199057,1895121000028733,26101019958503,2021-05-08 17:00:00,2021-05-24 17:00:00,2021-05-21 17:00:00,245703.0,26813.0,467003.0,248113.0,218890.0,3.202781e+04,156088.00,17113.00,21088,108.0,216.0,1.729801e+07,17558100.0,-2.600910e+05,20210524,2021-03-30 17:00:00,15.0,1.0,Y,218890.0,2500000.0,Active,20210524.0


In [15]:
df_mar

,Unnamed: 0,id,trx_time,mean_trx_time_interval,max_trx_time_interval,min_trx_time_interval,cashout_status,total_transaction,total_cashout,cashout_percentage,purchase,total_transaction_amount,total_cashout_transaction_amount,cashout_amount_percentage,cashout_plafond_percentage,trx_plafond_percentage,merchant,most_merchant,total_merchant_transaction,merchant_transaction_percentage,merchant_transaction_amount_percentage,merchant_non_cashout_percentage,merchant_non_cashout_amount_percentage,merchant_plafond_percentage,plafond,purchase_status,total_success_trx,success_trx_percentage,cashout_success_trx_percentage,non_cashout_success_trx_percentage,merchant_success_trx_percentage,cif_id,oda,acct_opn_date,age,detail_gender,detail_marital_status,detail_last_education,detail_occupation_type,grade,score,desc,dpd
0,0,1895121000014142,2021-03-03 22:42:44,NaN,NaN,NaN,False,1,0,0.0,1179125.0,1179125.0,0.0,0.000,0.000,5.896,tokopedia,tokopedia,1,100.0,100.000,100.0,100.0,5.896,20000000.0,SUCCESS,1,100.0,0.0,100.0,100.0,C000013866,1895121000014142,2021-03-03 17:00:00,33.0,Perempuan,Menikah,Sekolah Menengah Atas (SMA),Karyawan,5.0,699.0,APPROVE OVERRIDE,0
1,1,1895121000014144,2021-03-04 11:41:23,NaN,NaN,NaN,True,1,1,100.0,12000000.0,12000000.0,12000000.0,100.000,60.000,60.000,rekening bri,rekening bri,1,100.0,100.000,NaN,NaN,60.000,20000000.0,SUCCESS,1,100.0,100.0,0.0,100.0,C000013868,1895121000014144,2021-03-04 17:00:00,34.0,Laki-laki,Menikah,Strata 1 (S1),Wiraswasta,2.0,807.0,APPROVE,8
2,2,1895121000014145,2021-03-04 12:27:25,NaN,NaN,NaN,True,1,1,100.0,12000000.0,12000000.0,12000000.0,100.000,60.000,60.000,rekening bri,rekening bri,1,100.0,100.000,NaN,NaN,60.000,20000000.0,SUCCESS,1,100.0,100.0,0.0,100.0,C000013869,1895121000014145,2021-03-04 17:00:00,23.0,Perempuan,Lajang,Strata 1 (S1),Karyawan,4.0,744.0,APPROVE OVERRIDE,100
3,3,1895121000014128,2021-03-04 12:49:17,NaN,NaN,NaN,False,1,0,0.0,687347.0,687347.0,0.0,0.000,0.000,9.165,tokopedia,tokopedia,1,100.0,100.000,NaN,100.0,9.165,7500000.0,REVERSAL,0,0.0,0.0,0.0,0.0,C000013852,1895121000014128,2021-03-01 17:00:00,28.0,Laki-laki,Lajang,Strata 1 (S1),Profesional,2.0,808.0,APPROVE,0
4,4,1895121000014131,2021-03-04 13:18:43,71372.333,161307.0,3451.0,False,4,1,25.0,280256.0,11209082.0,10000000.0,89.213,50.000,56.045,tokopedia,tokopedia,3,75.0,10.787,100.0,100.0,6.045,20000000.0,SUCCESS,4,100.0,25.0,75.0,100.0,C000013855,1895121000014131,2021-03-01 17:00:00,49.0,Perempuan,Menikah,Sekolah Menengah Atas (SMA),Wiraswasta,7.0,597.0,APPROVE OVERRIDE,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45204,45204,1895121000022628,2021-04-04 17:16:50,583524.000,583524.0,583524.0,True,2,2,100.0,2000000.0,4000000.0,4000000.0,100.000,64.000,64.000,rekening bri,rekening bri,2,100.0,100.000,NaN,NaN,64.000,6250000.0,SUCCESS,2,100.0,100.0,0.0,100.0,C000022333,1895121000022628,2021-03-28 17:00:00,23.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Wiraswasta,4.0,651.0,APPROVE,0
45205,45205,1895121000022503,2021-04-04 17:17:32,434018.000,434018.0,434018.0,True,2,2,100.0,1300000.0,4675000.0,4675000.0,100.000,83.111,83.111,rekening bri,rekening bri,2,100.0,100.000,NaN,NaN,83.111,5625000.0,SUCCESS,2,100.0,100.0,0.0,100.0,C000022208,1895121000022503,2021-03-28 17:00:00,23.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Karyawan,5.0,628.0,APPROVE,44
45206,45206,1895121000027536,2021-04-04 17:18:01,57811.333,248699.0,98.0,True,7,7,100.0,50000.0,3630000.0,3630000.0,100.000,96.800,96.800,rekening bri,rekening bri,7,100.0,100.000,NaN,NaN,96.800,3750000.0,SUCCESS,7,100.0,100.0,0.0,100.0,C000027176,1895121000027536,2021-03-30 17:00:00,24.0,Perempuan,Menikah,Sekolah Menengah Atas (SMA),Lain-Lain,8.0,599.0,APPROVE,74
45207,45207,1895121000023450,2021-04-04 17:19:57,191576.000,571866.0,424.0,True,4,4,100.0,300000.0,6550000.0,6550000.0,100.000,95.273,95.273,rekening bri,rekening bri,4,100.0,100.000,NaN,N

In [16]:
df_label_maret = df_label_maret[df_label_maret['oda'].notna()]
df_label_maret['oda'] = df_label_maret['oda'].astype(np.int64)

In [17]:
df_label_maret

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd,collectibility,hold_flag,hold,limit,oda_status,posisi
0,99143153,1895121000014127,16501031342508,2021-03-23 17:00:00,2021-04-08 17:00:00,2021-04-03 17:00:00,1020232.0,1020232.0,1020232.0,1020232.0,0.0,2.272930e+07,25021783.34,20598519.99,22812000,182.0,936.0,2.048392e+08,180741990.0,2.409723e+07,20210408,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210408.0
1,99143153,1895121000014127,16501031342508,2021-04-23 17:00:00,2021-05-09 17:00:00,2021-05-04 17:00:00,1287272.0,1287272.0,2307504.0,2307504.0,0.0,2.100589e+07,22046691.34,18675339.34,21022370,270.0,972.0,2.601165e+08,320305941.0,-6.018941e+07,20210509,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210509.0
2,99143153,1895121000014127,16501031342508,2021-05-23 17:00:00,2021-06-08 17:00:00,2021-06-03 17:00:00,1287272.0,1287272.0,3594776.0,3594776.0,0.0,1.373584e+07,20029453.99,10478869.99,12293953,84.0,868.0,5.606709e+07,345155440.0,-2.890884e+08,20210608,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210608.0
3,99143154,1895121000014128,13901129110503,2021-03-15 17:00:00,2021-03-31 17:00:00,2021-04-15 17:00:00,0.0,0.0,0.0,0.0,0.0,4.608916e+05,10349926.60,49001.60,75528,112.0,420.0,6.753460e+07,361420500.0,-2.938859e+08,20210331,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210331.0
4,99143154,1895121000014128,13901129110503,2021-04-15 17:00:00,2021-05-01 17:00:00,2021-05-15 17:00:00,0.0,0.0,0.0,0.0,0.0,5.036160e+04,58501.60,48501.60,48502,25.0,75.0,7.500000e+06,7387500.0,1.125000e+05,20210501,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210501.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36854,99199056,1895121000028732,79901042281536,2021-05-08 17:00:00,2021-05-24 17:00:00,2021-06-08 17:00:00,2032670.0,0.0,3163296.0,0.0,3163296.0,6.663077e+05,1233641.00,83641.00,683641,108.0,81.0,4.603500e+07,16416000.0,2.961900e+07,20210524,2021-03-30 17:00:00,45.0,2.0,N,0.0,6250000.0,Active,20210524.0
36855,99199056,1895121000028732,79901042281536,2021-06-08 17:00:00,2021-06-24 17:00:00,2021-06-08 17:00:00,2961776.0,0.0,6125072.0,0.0,6125072.0,8.938804e+04,95626.00,87626.00,87626,27.0,135.0,2.295000e+03,29729700.0,-2.972740e+07,20210622,2021-03-30 17:00:00,74.0,2.0,N,0.0,6250000.0,Active,20210622.0
36856,99199057,1895121000028733,26101019958503,2021-04-08 17:00:00,2021-04-24 17:00:00,2021-05-01 17:00:00,221300.0,221300.0,221300.0,221300.0,0.0,1.299460e+05,1566946.00,16946.00,20946,56.0,196.0,7.700000e+07,75894000.0,1.106000e+06,20210424,2021-03-30 17:00:00,15.0,1.0,Y,221300.0,2500000.0,Active,20210424.0
36857,99199057,1895121000028733,26101019958503,2021-05-08 17:00:00,2021-05-24 17:00:00,2021-05-21 17:00:00,245703.0,26813.0,467003.0,248113.0,218890.0,3.202781e+04,156088.00,17113.00,21088,108.0,216.0,1.729801e+07,17558100.0,-2.600910e+05,20210524,2021-03-30 17:00:00,15.0,1.0,Y,218890.0,2500000.0,Active,20210524.0


In [18]:
#select oda with validasi_dok = not ok
oda_fraud = df_fraud.loc[df_fraud['validasi_dok'] == 'NOT OK']
oda_fraud

,oda,autodebet_status,limit,no_hp,active_no_hp,validasi_dok
15,1895121000017886,jalan,"2,500,000",8.533994e+10,AKTIF,NOT OK
19,1895121000018729,pending,"6,875,000",8.782696e+10,AKTIF,NOT OK
20,1895121000018733,jalan,"2,500,000",8.128428e+10,AKTIF,NOT OK
21,1895121000018938,pending,"6,250,000",8.177004e+10,AKTIF,NOT OK
24,1895121000020527,jalan,"3,750,000",8.572763e+10,TIDAK AKTIF,NOT OK
...,...,...,...,...,...,...
33373,1895121000043133,pending,"5,000,000",8.157326e+10,NaN,NOT OK
33374,1895121000041285,pending,"3,750,000",8.228103e+10,AKTIF,NOT OK
33375,1895121000045773,pending,"1,250,000",8.228537e+10,NaN,NOT OK
33378,1895121000046169,pending,"6,875,000",8.227990e+10,NaN,NOT OK


In [19]:
#Hapus nilai Inf data train

list_column = df_bill.columns.to_list()
list_inf_train    = []

for i in range(df_bill.shape[1]):
    list_inf_train.append(df_bill[list_column[i]].max())
    
inf_list_train = pd.DataFrame(
    {'Fitur': list_column,
     'Inf ?': list_inf_train,
    })

In [20]:
df_mar= df_mar[['cif_id','oda','age','detail_gender','detail_marital_status','detail_last_education','detail_occupation_type',
    'grade','score','dpd']]

df_mar.columns.tolist()

['cif_id',
 'oda',
 'age',
 'detail_gender',
 'detail_marital_status',
 'detail_last_education',
 'detail_occupation_type',
 'grade',
 'score',
 'dpd']

# Filter Oda pada table billing untuk akun ceria buka dibulan Maret 2021

In [21]:
list_oda_maret = df_mar.oda.unique().tolist()
list_oda_fraud =oda_fraud.oda.unique().tolist()

In [22]:
len(list_oda_fraud)

10818

In [23]:
df_label_maret=df_label_maret[df_label_maret['oda'].isin(list_oda_maret)]
print(df_label_maret.oda.nunique())
df_label_maret

12641


,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd,collectibility,hold_flag,hold,limit,oda_status,posisi
0,99143153,1895121000014127,16501031342508,2021-03-23 17:00:00,2021-04-08 17:00:00,2021-04-03 17:00:00,1020232.0,1020232.0,1020232.0,1020232.0,0.0,2.272930e+07,25021783.34,20598519.99,22812000,182.0,936.0,2.048392e+08,180741990.0,2.409723e+07,20210408,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210408.0
1,99143153,1895121000014127,16501031342508,2021-04-23 17:00:00,2021-05-09 17:00:00,2021-05-04 17:00:00,1287272.0,1287272.0,2307504.0,2307504.0,0.0,2.100589e+07,22046691.34,18675339.34,21022370,270.0,972.0,2.601165e+08,320305941.0,-6.018941e+07,20210509,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210509.0
2,99143153,1895121000014127,16501031342508,2021-05-23 17:00:00,2021-06-08 17:00:00,2021-06-03 17:00:00,1287272.0,1287272.0,3594776.0,3594776.0,0.0,1.373584e+07,20029453.99,10478869.99,12293953,84.0,868.0,5.606709e+07,345155440.0,-2.890884e+08,20210608,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210608.0
3,99143154,1895121000014128,13901129110503,2021-03-15 17:00:00,2021-03-31 17:00:00,2021-04-15 17:00:00,0.0,0.0,0.0,0.0,0.0,4.608916e+05,10349926.60,49001.60,75528,112.0,420.0,6.753460e+07,361420500.0,-2.938859e+08,20210331,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210331.0
4,99143154,1895121000014128,13901129110503,2021-04-15 17:00:00,2021-05-01 17:00:00,2021-05-15 17:00:00,0.0,0.0,0.0,0.0,0.0,5.036160e+04,58501.60,48501.60,48502,25.0,75.0,7.500000e+06,7387500.0,1.125000e+05,20210501,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210501.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36854,99199056,1895121000028732,79901042281536,2021-05-08 17:00:00,2021-05-24 17:00:00,2021-06-08 17:00:00,2032670.0,0.0,3163296.0,0.0,3163296.0,6.663077e+05,1233641.00,83641.00,683641,108.0,81.0,4.603500e+07,16416000.0,2.961900e+07,20210524,2021-03-30 17:00:00,45.0,2.0,N,0.0,6250000.0,Active,20210524.0
36855,99199056,1895121000028732,79901042281536,2021-06-08 17:00:00,2021-06-24 17:00:00,2021-06-08 17:00:00,2961776.0,0.0,6125072.0,0.0,6125072.0,8.938804e+04,95626.00,87626.00,87626,27.0,135.0,2.295000e+03,29729700.0,-2.972740e+07,20210622,2021-03-30 17:00:00,74.0,2.0,N,0.0,6250000.0,Active,20210622.0
36856,99199057,1895121000028733,26101019958503,2021-04-08 17:00:00,2021-04-24 17:00:00,2021-05-01 17:00:00,221300.0,221300.0,221300.0,221300.0,0.0,1.299460e+05,1566946.00,16946.00,20946,56.0,196.0,7.700000e+07,75894000.0,1.106000e+06,20210424,2021-03-30 17:00:00,15.0,1.0,Y,221300.0,2500000.0,Active,20210424.0
36857,99199057,1895121000028733,26101019958503,2021-05-08 17:00:00,2021-05-24 17:00:00,2021-05-21 17:00:00,245703.0,26813.0,467003.0,248113.0,218890.0,3.202781e+04,156088.00,17113.00,21088,108.0,216.0,1.729801e+07,17558100.0,-2.600910e+05,20210524,2021-03-30 17:00:00,15.0,1.0,Y,218890.0,2500000.0,Active,20210524.0


# Exclude oda fraud

In [24]:
df_label_maret = df_label_maret[~df_label_maret['oda'].isin(list_oda_fraud)]
df_label_maret

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd,collectibility,hold_flag,hold,limit,oda_status,posisi
0,99143153,1895121000014127,16501031342508,2021-03-23 17:00:00,2021-04-08 17:00:00,2021-04-03 17:00:00,1020232.0,1020232.0,1020232.0,1020232.0,0.0,2.272930e+07,25021783.34,20598519.99,22812000,182.0,936.0,2.048392e+08,180741990.0,2.409723e+07,20210408,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210408.0
1,99143153,1895121000014127,16501031342508,2021-04-23 17:00:00,2021-05-09 17:00:00,2021-05-04 17:00:00,1287272.0,1287272.0,2307504.0,2307504.0,0.0,2.100589e+07,22046691.34,18675339.34,21022370,270.0,972.0,2.601165e+08,320305941.0,-6.018941e+07,20210509,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210509.0
2,99143153,1895121000014127,16501031342508,2021-05-23 17:00:00,2021-06-08 17:00:00,2021-06-03 17:00:00,1287272.0,1287272.0,3594776.0,3594776.0,0.0,1.373584e+07,20029453.99,10478869.99,12293953,84.0,868.0,5.606709e+07,345155440.0,-2.890884e+08,20210608,2021-03-01 17:00:00,0.0,1.0,N,0.0,9000000.0,Active,20210608.0
3,99143154,1895121000014128,13901129110503,2021-03-15 17:00:00,2021-03-31 17:00:00,2021-04-15 17:00:00,0.0,0.0,0.0,0.0,0.0,4.608916e+05,10349926.60,49001.60,75528,112.0,420.0,6.753460e+07,361420500.0,-2.938859e+08,20210331,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210331.0
4,99143154,1895121000014128,13901129110503,2021-04-15 17:00:00,2021-05-01 17:00:00,2021-05-15 17:00:00,0.0,0.0,0.0,0.0,0.0,5.036160e+04,58501.60,48501.60,48502,25.0,75.0,7.500000e+06,7387500.0,1.125000e+05,20210501,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210501.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36836,99199049,1895121000028725,404401015050539,2021-05-08 17:00:00,2021-05-24 17:00:00,2021-05-19 17:00:00,452417.0,10000.0,1131071.0,688654.0,442417.0,4.869459e+04,275000.00,25000.00,35000,135.0,297.0,2.295016e+07,38504862.0,-1.555470e+07,20210524,2021-03-30 17:00:00,15.0,1.0,Y,442417.0,5000000.0,Active,20210524.0
36837,99199049,1895121000028725,404401015050539,2021-06-08 17:00:00,2021-06-24 17:00:00,2021-06-08 17:00:00,904343.0,0.0,2035414.0,688654.0,1346760.0,2.870370e+04,35000.00,25000.00,25000,NaN,54.0,NaN,270000.0,-2.700000e+05,20210622,2021-03-30 17:00:00,44.0,2.0,Y,904343.0,5000000.0,Active,20210622.0
36856,99199057,1895121000028733,26101019958503,2021-04-08 17:00:00,2021-04-24 17:00:00,2021-05-01 17:00:00,221300.0,221300.0,221300.0,221300.0,0.0,1.299460e+05,1566946.00,16946.00,20946,56.0,196.0,7.700000e+07,75894000.0,1.106000e+06,20210424,2021-03-30 17:00:00,15.0,1.0,Y,221300.0,2500000.0,Active,20210424.0
36857,99199057,1895121000028733,26101019958503,2021-05-08 17:00:00,2021-05-24 17:00:00,2021-05-21 17:00:00,245703.0,26813.0,467003.0,248113.0,218890.0,3.202781e+04,156088.00,17113.00,21088,108.0,216.0,1.729801e+07,17558100.0,-2.600910e+05,20210524,2021-03-30 17:00:00,15.0,1.0,Y,218890.0,2500000.0,Active,20210524.0


# Take demography data for specific ODA opened in march

In [29]:
list_oda_maret_without_fraud = df_label_maret.oda.unique().tolist()

#take oda in table df_mar based on list_oda_maret_without_fraud
df_mar=df_mar[df_mar['oda'].isin(list_oda_maret_without_fraud)]
df_mar = df_mar.sort_values('score').drop_duplicates('oda', keep='last')
df_mar.shape

(9468, 10)

In [30]:
df_mar.head()

,cif_id,oda,age,detail_gender,detail_marital_status,detail_last_education,detail_occupation_type,grade,score,dpd
2175,C000013928,1895121000014204,39.0,Laki-laki,Duda/Janda,Diploma 3 (D3),Profesional,8.0,442.0,7
19736,C000020521,1895121000020805,33.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Karyawan,9.0,490.0,74
43683,C000027318,1895121000027679,46.0,Laki-laki,Menikah,Strata 1 (S1),Lain-Lain,9.0,490.0,13
41155,C000026778,1895121000027132,34.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Karyawan,9.0,490.0,74
2606,C000021562,1895121000021854,26.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Lain-Lain,9.0,490.0,0


In [31]:
def drop_duplicate(table_billing):
    table_billing = table_billing.sort_values('ds_ts').drop_duplicates('oda', keep='last')
    return table_billing

# 1.Tabel billing maret

In [42]:
df_bill_maret=df_label_maret.loc[(df_label_maret['ds_ts'] >= 20210301) & (df_label_maret['ds_ts'] <= 20210331)]
df_bill_maret.head()

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd,collectibility,hold_flag,hold,limit,oda_status,posisi
3,99143154,1895121000014128,13901129110503,2021-03-15 17:00:00,2021-03-31 17:00:00,2021-04-15 17:00:00,0.0,0.0,0.0,0.0,0.0,4.608916e+05,1.034993e+07,49001.60,75528,112.0,420.0,6.753460e+07,3.614205e+08,-2.938859e+08,20210331,2021-03-01 17:00:00,0.0,1.0,N,0.0,7500000.0,Active,20210331.0
26,99144554,1895121000014139,12701007172539,2021-03-14 17:00:00,2021-03-30 17:00:00,2021-03-25 17:00:00,275198.0,275198.0,275198.0,275198.0,0.0,1.458128e+07,1.505338e+07,6181385.00,15049885,28.0,252.0,9.920400e+04,3.983585e+07,-3.973665e+07,20210330,2021-03-03 17:00:00,0.0,1.0,N,0.0,1500000.0,Active,20210330.0
31,99144561,1895121000014142,138601000862538,2021-03-04 17:00:00,2021-03-20 17:00:00,2021-03-15 17:00:00,1193275.0,1193275.0,1193275.0,1193275.0,0.0,3.238778e+07,3.577775e+07,30674462.00,30874962,25.0,225.0,1.785000e+05,1.277607e+08,-1.275822e+08,20210320,2021-03-03 17:00:00,0.0,1.0,N,0.0,20000000.0,Active,20210320.0
36,99145123,1895121000014144,13201000788560,2021-03-13 17:00:00,2021-03-29 17:00:00,2021-03-25 17:00:00,1780400.0,1780400.0,1780400.0,1780400.0,0.0,8.890762e+07,1.138305e+08,58799023.80,73813524,84.0,336.0,6.180639e+08,1.821519e+09,-1.203455e+09,20210329,2021-03-04 17:00:00,0.0,1.0,N,0.0,20000000.0,Active,20210329.0
40,99145124,1895121000014145,335201040892534,2021-03-13 17:00:00,2021-03-29 17:00:00,2021-04-13 17:00:00,6559400.0,0.0,6559400.0,0.0,6559400.0,3.808002e+06,1.999976e+07,87407.11,2488781,1092.0,1904.0,1.789993e+09,1.701066e+09,8.892615e+07,20210329,2021-03-04 17:00:00,15.0,1.0,Y,6559400.0,20000000.0,Active,20210329.0


In [54]:
df_bill_maret=drop_duplicate(df_bill_maret)
df_bill_maret.head()

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd,collectibility,hold_flag,hold,limit,oda_status,posisi
31,99144561,1895121000014142,138601000862538,2021-03-04 17:00:00,2021-03-20 17:00:00,2021-03-15 17:00:00,1193275.0,1193275.0,1193275.0,1193275.0,0.0,3.238778e+07,35777750.00,30674462.00,30874962,25.0,225.0,1.785000e+05,1.277607e+08,-127582200.0,20210320,2021-03-03 17:00:00,0.0,1.0,N,0.0,20000000.0,Active,20210320.0
75,99146107,1895121000014158,683001000436503,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-03-24 17:00:00,442600.0,442600.0,442600.0,442600.0,0.0,5.136618e+05,3133847.20,34456.20,49388,50.0,450.0,9.323650e+07,9.112500e+07,2111500.0,20210324,2021-03-06 17:00:00,16.0,1.0,Y,442600.0,5000000.0,Active,20210324.0
99,99146500,1895121000014167,1801049973504,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-03-19 17:00:00,3203775.0,3203775.0,3203775.0,3203775.0,0.0,1.056295e+07,10870318.33,9383194.33,10544349,25.0,225.0,6.797500e+04,9.496090e+07,-94892925.0,20210324,2021-03-07 17:00:00,0.0,1.0,N,0.0,20000000.0,Active,20210324.0
86,99146111,1895121000014162,60901007395503,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-03-19 17:00:00,1337800.0,1337800.0,1337800.0,1337800.0,0.0,2.767492e+06,10049420.47,57120.47,2140224,475.0,1425.0,9.460984e+08,8.235412e+08,122557150.0,20210324,2021-03-06 17:00:00,0.0,1.0,Y,660528.0,16500000.0,Active,20210324.0
68,99145633,1895121000014155,723001003234535,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-03-16 17:00:00,350623.0,350623.0,350623.0,350623.0,0.0,1.036299e+07,16400614.20,1701350.40,9709615,350.0,725.0,3.444835e+09,3.086067e+09,358767545.0,20210324,2021-03-05 17:00:00,0.0,1.0,N,0.0,20000000.0,Active,20210324.0


### 1.1 Join with demography data

In [55]:
dfinal_maret = df_bill_maret.merge(df_mar, on="oda", how = 'inner')
dfinal_maret.shape

(17, 38)

In [56]:
#Labeling NPL & PL per ODA
label_maret = []
for i in range(dfinal_maret.shape[0]):
    if dfinal_maret.dpd_x.iloc[i] >= 30:
        label_maret.append(1) #NPL
    else :
        label_maret.append(0) #PL
        
dfinal_maret['label'] = label_maret

# Drop dpd column and save it
dfinal_maret=dfinal_maret.drop(columns=['dpd_x','dpd_y'])
dfinal_maret.to_csv('data_billing_maret_new_dpd.csv')

In [57]:
dfinal_maret

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,collectibility,hold_flag,hold,limit,oda_status,posisi,cif_id,age,detail_gender,detail_marital_status,detail_last_education,detail_occupation_type,grade,score,label
0,99144561,1895121000014142,138601000862538,2021-03-04 17:00:00,2021-03-20 17:00:00,2021-03-15 17:00:00,1193275.0,1193275.0,1193275.0,1193275.0,0.0,3.238778e+07,3.577775e+07,30674462.00,30874962,25.0,225.0,1.785000e+05,1.277607e+08,-1.275822e+08,20210320,2021-03-03 17:00:00,1.0,N,0.0,20000000.0,Active,20210320.0,C000013866,33.0,Perempuan,Menikah,Sekolah Menengah Atas (SMA),Karyawan,5.0,699.0,0
1,99146107,1895121000014158,683001000436503,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-03-24 17:00:00,442600.0,442600.0,442600.0,442600.0,0.0,5.136618e+05,3.133847e+06,34456.20,49388,50.0,450.0,9.323650e+07,9.112500e+07,2.111500e+06,20210324,2021-03-06 17:00:00,1.0,Y,442600.0,5000000.0,Active,20210324.0,C000013882,26.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Karyawan,2.0,804.0,0
2,99146500,1895121000014167,1801049973504,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-03-19 17:00:00,3203775.0,3203775.0,3203775.0,3203775.0,0.0,1.056295e+07,1.087032e+07,9383194.33,10544349,25.0,225.0,6.797500e+04,9.496090e+07,-9.489292e+07,20210324,2021-03-07 17:00:00,1.0,N,0.0,20000000.0,Active,20210324.0,C000013891,54.0,Laki-laki,Menikah,Strata 1 (S1),Wiraswasta,4.0,743.0,0
3,99146111,1895121000014162,60901007395503,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-03-19 17:00:00,1337800.0,1337800.0,1337800.0,1337800.0,0.0,2.767492e+06,1.004942e+07,57120.47,2140224,475.0,1425.0,9.460984e+08,8.235412e+08,1.225572e+08,20210324,2021-03-06 17:00:00,1.0,Y,660528.0,16500000.0,Active,20210324.0,C000013886,33.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Karyawan,6.0,688.0,0
4,99145633,1895121000014155,723001003234535,2021-03-08 17:00:00,2021-03-24 17:00:00,2021-03-16 17:00:00,350623.0,350623.0,350623.0,350623.0,0.0,1.036299e+07,1.640061e+07,1701350.40,9709615,350.0,725.0,3.444835e+09,3.086067e+09,3.587675e+08,20210324,2021-03-05 17:00:00,1.0,N,0.0,20000000.0,Active,20210324.0,C000013879,31.0,Laki-laki,Menikah,Strata 1 (S1),Karyawan,4.0,732.0,0
5,99145125,1895121000014146,55301028719505,2021-03-11 17:00:00,2021-03-27 17:00:00,2021-03-12 17:00:00,591735.0,591735.0,591735.0,591735.0,0.0,8.238794e+06,9.228178e+06,7263978.00,8131312,108.0,810.0,6.236176e+07,8.577978e+07,-2.341802e+07,20210327,2021-03-04 17:00:00,1.0,N,0.0,3000000.0,Active,20210327.0,C000013870,30.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Wiraswasta,2.0,797.0,0
6,99146108,1895121000014159,66001022703505,2021-03-12 17:00:00,2021-03-28 17:00:00,2021-03-15 17:00:00,167872.0,167872.0,167872.0,167872.0,0.0,8.157991e+07,8.398046e+07,80469206.00,82249796,56.0,308.0,9.879360e+07,5.460221e+07,4.419139e+07,20210328,2021-03-06 17:00:00,1.0,N,0.0,13500000.0,Active,20210328.0,C000013883,30.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Wiraswasta,6.0,640.0,0
7,99147817,1895121000014175,52901013617501,2021-03-12 17:00:00,2021-03-28 17:00:00,2021-03-23 17:00:00,1782158.0,1782158.0,1782158.0,1782158.0,0.0,2.577695e+07,2.764362e+07,24129921.20,24822015,84.0,252.0,1.152614e+08,4.935216e+07,6.590928e+07,20210328,2021-03-10 17:00:00,1.0,N,0.0,4500000.0,Active,20210328.0,C000013899,45.0,Perempuan,Menikah,Strata 1 (S1),Karyawan,5.0,697.0,0
8,99146499,1895121000014166,343601030331538,2021-03-12 17:00:00,2021-03-28 17:00:00,2021-03-23 17:00:00,196675.0,196675.0,196675.0,196675.0,0.0,1.857293e+06,4.551459e+06,941385.80,1480013,140.0,1316.0,2.572589e+08,3.725229e+08,-1.152640e+08,20210328,2021-03-07 17:00:00,1.0,N,0.0,10000000.0,Active,20210328.0,C000013890,36.0,Laki-laki,Menikah,Strata 1 (S1),Lain-Lain,4.0,739.0,0
9,99145124,1895121000014145,335201040892534,2021-03-13 17:00:00,2021-03-29 17:00:00,202

# 2.Tabel billing april

In [121]:
df_bill_april = df_label_maret.loc[(df_label_maret['ds_ts'] >= 20210401) & (df_label_maret['ds_ts'] <= 20210430)]
df_bill_april.shape

(9222, 29)

In [122]:
df_bill_april.oda.nunique()

9222

In [123]:
df_bill_april=drop_duplicate(df_bill_april)

### 2.1 Join with demography data

In [124]:
dfinal_april = df_bill_april.merge(df_mar, on="oda", how = 'inner')
dfinal_april

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd_x,collectibility,hold_flag,hold,limit,oda_status,posisi,cif_id,age,detail_gender,detail_marital_status,detail_last_education,detail_occupation_type,grade,score,dpd_y
0,99147351,1895121000014173,202201005078507,2021-03-16 17:00:00,2021-04-01 17:00:00,2021-03-28 17:00:00,19610751.0,19610751.0,19610751.0,19610751.0,0.0,1.863760e+07,92261029.00,619588.00,13592398,216.0,1431.0,2.717562e+09,2.950181e+09,-232619094.0,20210401,2021-03-09 17:00:00,0.0,1.0,N,0.0,20000000.0,Active,20210401.0,C000013897,32.0,Perempuan,Menikah,Strata 2 (S2),Lain-Lain,3.0,761.0,5
1,99143902,1895121000014132,617401028725530,2021-03-16 17:00:00,2021-04-01 17:00:00,2021-03-28 17:00:00,652134.0,652134.0,652134.0,652134.0,0.0,1.777416e+07,20440727.40,2440727.40,20440728,27.0,108.0,9.242100e+04,4.862345e+08,-486142074.0,20210401,2021-03-02 17:00:00,0.0,1.0,N,0.0,20000000.0,Active,20210401.0,C000013856,37.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Lain-Lain,7.0,608.0,5
2,99144562,1895121000014143,15801038925500,2021-03-16 17:00:00,2021-04-01 17:00:00,2021-03-28 17:00:00,350623.0,350623.0,350623.0,350623.0,0.0,9.073346e+05,5463976.35,115976.35,364477,459.0,594.0,5.413664e+08,5.181705e+08,23195943.0,20210401,2021-03-03 17:00:00,0.0,1.0,N,0.0,20000000.0,Active,20210401.0,C000013867,32.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Lain-Lain,7.0,563.0,5
3,99150458,1895121000014325,701401009164538,2021-03-17 17:00:00,2021-04-02 17:00:00,2021-04-05 17:00:00,2770427.0,2770427.0,2770427.0,2770427.0,0.0,1.122130e+05,2665625.00,13125.00,13650,54.0,162.0,1.208486e+08,1.208628e+08,-14175.0,20210402,2021-03-17 17:00:00,15.0,1.0,Y,2770427.0,6875000.0,Active,20210402.0,C000014066,32.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Karyawan,3.0,700.0,4
4,99150393,1895121000014277,207401009663501,2021-03-17 17:00:00,2021-04-02 17:00:00,2021-03-29 17:00:00,676504.0,676504.0,676504.0,676504.0,0.0,2.295570e+05,1658352.00,38952.00,58601,486.0,1134.0,3.719190e+08,3.793149e+08,-7395894.0,20210402,2021-03-16 17:00:00,0.0,1.0,N,0.0,1250000.0,Active,20210402.0,C000014012,21.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Lain-Lain,9.0,490.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9217,99150035,1895121000014267,409901021724537,2021-04-14 17:00:00,2021-04-30 17:00:00,2021-05-14 17:00:00,1851065.0,0.0,1851065.0,0.0,1851065.0,2.226880e+05,1763662.00,50125.00,55625,250.0,725.0,1.085027e+08,1.364884e+08,-27985700.0,20210430,2021-03-16 17:00:00,15.0,1.0,Y,1851065.0,2500000.0,Active,20210430.0,C000013998,24.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Karyawan,9.0,567.0,68
9218,99149971,1895121000014204,420301000164505,2021-04-14 17:00:00,2021-04-30 17:00:00,2021-04-25 17:00:00,3009198.0,3009198.0,3009198.0,3009198.0,0.0,2.969714e+06,13238704.00,507373.00,2596873,375.0,1100.0,7.983884e+08,8.343600e+08,-35971625.0,20210430,2021-03-16 17:00:00,0.0,1.0,N,0.0,17000000.0,Active,20210430.0,C000013928,39.0,Laki-laki,Duda/Janda,Diploma 3 (D3),Profesional,8.0,442.0,7
9219,99150953,1895121000014401,105301012665503,2021-04-14 17:00:00,2021-04-30 17:00:00,2021-04-25 17:00:00,1592920.0,1592920.0,1592920.0,1592920.0,0.0,4.382200e+04,43822.00,43822.00,43822,175.0,225.0,3.050000e+08,3.050000e+08,0.0,20210430,2021-03-18 17:00:00,0.0,1.0,N,0.0,6875000.0,Active,20210430.0,C000014125,37.0,Laki-laki,Menikah,Sekolah Menengah Pertama (SMP),Lain-Lain,3.0,691.0,7
9220,99157417,1895121000016694,5101105404502,2021-04-14 17:00:00,2021-04-30 17:00:00,2021-04-26 17:00:00,1558995.0,1558995.0,1558995.0,1558995.0,0.0,9.945178e+05,8081747.80,54162.80,342207,375.0,1625.0,5.115991e+08,4.604336e+08,51165500.0,20210430,2021-03-22 17:00:00,0.0,1.0,Y,5

In [127]:
#Labeling NPL & PL per ODA
label_april = []
for i in range(dfinal_april.shape[0]):
    if dfinal_april.dpd_x.iloc[i] >= 30:
        label_april.append(1) #NPL
    else :
        label_april.append(0) #PL
        
dfinal_april['label'] = label_april

In [128]:
# Drop dpd column and save it
dfinal_april=dfinal_april.drop(columns=['dpd_x','dpd_y'])
dfinal_april.to_csv('data_billing_april_new_dpd.csv')

# 3.Tabel billing Mei

In [129]:
df_bill_mei=df_label_maret.loc[(df_label_maret['ds_ts'] >= 20210501) & (df_label_maret['ds_ts'] <= 20210531)]
df_bill_mei.shape

(9365, 29)

In [130]:
df_bill_mei.oda.nunique()

9345

In [131]:
df_bill_mei=drop_duplicate(df_bill_mei)

In [132]:
df_bill_mei.shape

(9345, 29)

### 3.1 Join with demography data

In [133]:
dfinal_mei = df_bill_mei.merge(df_mar, on="oda", how = 'inner')
dfinal_mei

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd_x,collectibility,hold_flag,hold,limit,oda_status,posisi,cif_id,age,detail_gender,detail_marital_status,detail_last_education,detail_occupation_type,grade,score,dpd_y
0,99154667,1895121000015596,91001051607539,2021-04-16 17:00:00,2021-05-02 17:00:00,2021-04-27 17:00:00,1005086.0,1005086.0,1005086.0,1005086.0,0.0,7.946145e+05,4917405.00,42405.00,72804,208.0,832.0,1.999425e+08,1.993849e+08,557648.0,20210502,2021-03-21 17:00:00,0.0,1.0,NaN,NaN,6875000.0,Active,20210502.0,C000015336,21.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Karyawan,3.0,687.0,5
1,99151652,1895121000014725,414101016208539,2021-04-16 17:00:00,2021-05-02 17:00:00,2021-04-27 17:00:00,154796.0,154796.0,154796.0,154796.0,0.0,4.268041e+07,45714097.00,38897505.00,43402268,572.0,1300.0,4.596096e+08,5.935808e+08,-133971214.0,20210502,2021-03-19 17:00:00,0.0,1.0,Y,4353400.0,3750000.0,Active,20210502.0,C000014453,34.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Wiraswasta,8.0,599.0,0
2,99157283,1895121000016561,187001003272507,2021-04-16 17:00:00,2021-05-02 17:00:00,2021-04-27 17:00:00,2655000.0,2655000.0,2655000.0,2655000.0,0.0,2.756848e+06,22014406.00,49055.00,1316795,780.0,2626.0,2.687302e+09,2.661073e+09,26229450.0,20210502,2021-03-22 17:00:00,0.0,1.0,N,0.0,5625000.0,Active,20210502.0,C000016281,28.0,Perempuan,Menikah,Strata 1 (S1),Karyawan,5.0,630.0,0
3,99147351,1895121000014173,202201005078507,2021-04-16 17:00:00,2021-05-02 17:00:00,2021-04-28 17:00:00,8814654.0,8814654.0,28425405.0,28425405.0,0.0,1.106379e+06,3137173.00,79850.00,779390,390.0,1326.0,1.501973e+09,1.533072e+09,-31099432.0,20210502,2021-03-09 17:00:00,0.0,1.0,NaN,NaN,20000000.0,Active,20210502.0,C000013897,32.0,Perempuan,Menikah,Strata 2 (S2),Lain-Lain,3.0,761.0,5
4,99168371,1895121000019785,679901017344531,2021-04-16 17:00:00,2021-05-02 17:00:00,2021-04-27 17:00:00,115434.0,115434.0,115434.0,115434.0,0.0,1.798197e+06,3053180.01,1053180.01,1748135,208.0,1040.0,2.055380e+08,2.160897e+08,-10551736.0,20210502,2021-03-26 17:00:00,0.0,1.0,NaN,NaN,6875000.0,Active,20210502.0,C000019505,33.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Karyawan,3.0,673.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9340,99154967,1895121000015896,700701010948534,2021-05-15 17:00:00,2021-05-31 17:00:00,2021-05-26 17:00:00,652139.0,652139.0,1436778.0,1436778.0,0.0,1.127798e+06,6881017.00,25000.00,567663,406.0,2900.0,8.593189e+08,8.604050e+08,-1086137.0,20210531,2021-03-21 17:00:00,0.0,1.0,N,0.0,3750000.0,Active,20210531.0,C000015635,35.0,Laki-laki,Menikah,Strata 1 (S1),Lain-Lain,8.0,597.0,6
9341,99158824,1895121000016858,12001057115505,2021-05-15 17:00:00,2021-05-31 17:00:00,2021-05-15 17:00:00,0.0,0.0,126857.0,126857.0,0.0,2.036552e+06,9913624.00,55641.00,761450,435.0,1827.0,2.957127e+09,2.950476e+09,6651121.0,20210531,2021-03-22 17:00:00,0.0,1.0,N,0.0,5000000.0,Active,20210531.0,C000016576,32.0,Laki-laki,Menikah,Strata 1 (S1),Karyawan,6.0,620.0,0
9342,99154981,1895121000015910,702301015974506,2021-05-15 17:00:00,2021-05-31 17:00:00,2021-05-15 17:00:00,0.0,0.0,2655000.0,2655000.0,0.0,1.195615e+06,3088635.40,50000.40,495241,232.0,1566.0,6.322138e+08,8.355153e+08,-203301455.0,20210531,2021-03-21 17:00:00,0.0,1.0,N,0.0,4375000.0,Active,20210531.0,C000015649,38.0,Perempuan,Menikah,Strata 1 (S1),Wiraswasta,7.0,604.0,0
9343,99151719,1895121000014792,427401025994532,2021-05-15 17:00:00,2021-05-31 17:00:00,2021-06-15 17:00:00,1185333.0,0.0,1860374.0,0.0,1860374.0,2.001724e+05,1545000.00,45000.00,45000,NaN,29.0,NaN,4.350000e+07,-43500000.0,20210531,2021-03-19 17:00:00,45.0,2.0,Y,1185333.0,4375000.0,Active,20210531.0,C000014514,27.0,Perempuan,Lajang,Sekolah 

In [134]:
#Labeling NPL & PL per ODA
label_mei = []
for i in range(dfinal_mei.shape[0]):
    if dfinal_mei.dpd_x.iloc[i] >= 30:
        label_mei.append(1) #NPL
    else :
        label_mei.append(0) #PL
        
dfinal_mei['label'] = label_mei

In [135]:
# Drop dpd column and save it
dfinal_mei=dfinal_mei.drop(columns=['dpd_x','dpd_y'])
dfinal_mei.to_csv('data_billing_mei_new_dpd.csv')

# 4.Tabel billing Juni

In [137]:
df_bill_juni=df_label_maret.loc[(df_label_maret['ds_ts'] >= 20210601) & (df_label_maret['ds_ts'] <= 20210630)]
df_bill_juni.shape

(8895, 29)

In [138]:
df_bill_juni.oda.nunique()

8833

In [139]:
df_bill_juni=drop_duplicate(df_bill_juni)

### 4.1 Join with demography data

In [140]:
dfinal_juni = df_bill_juni.merge(df_mar, on="oda", how = 'inner')
dfinal_juni

,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,dpd_x,collectibility,hold_flag,hold,limit,oda_status,posisi,cif_id,age,detail_gender,detail_marital_status,detail_last_education,detail_occupation_type,grade,score,dpd_y
0,99151652,1895121000014725,414101016208539,2021-05-16 17:00:00,2021-06-01 17:00:00,2021-05-16 17:00:00,0.0,0.0,154796.0,154796.0,0.0,1.950285e+07,39096505.0,6963458.0,10756902,406.0,1769.0,2.605160e+08,1.078218e+09,-817701487.0,20210601,2021-03-19 17:00:00,0.0,1.0,Y,4353400.0,3750000.0,Active,20210601.0,C000014453,34.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Wiraswasta,8.0,599.0,0
1,99157283,1895121000016561,187001003272507,2021-05-16 17:00:00,2021-06-01 17:00:00,2021-05-16 17:00:00,0.0,0.0,2655000.0,2655000.0,0.0,4.230447e+06,11794569.0,166140.0,4238900,783.0,3567.0,3.461164e+09,3.330742e+09,130421729.0,20210601,2021-03-22 17:00:00,0.0,1.0,N,0.0,5625000.0,Active,20210601.0,C000016281,28.0,Perempuan,Menikah,Strata 1 (S1),Karyawan,5.0,630.0,0
2,99152962,1895121000015311,752101007482533,2021-05-17 17:00:00,2021-06-02 17:00:00,2021-05-17 17:00:00,0.0,0.0,3186000.0,3186000.0,0.0,1.651359e+06,6030367.0,39127.0,440127,116.0,261.0,2.117070e+08,1.217130e+08,89993960.0,20210602,2021-03-20 17:00:00,0.0,1.0,N,0.0,3750000.0,Active,20210602.0,C000015043,27.0,Perempuan,Lajang,Sekolah Menengah Atas (SMA),Lain-Lain,8.0,588.0,0
3,99151468,1895121000014541,39401033724503,2021-05-17 17:00:00,2021-06-02 17:00:00,2021-05-17 17:00:00,0.0,0.0,640858.0,640858.0,0.0,8.108323e+05,2156728.0,13586.0,652689,2204.0,2726.0,4.644244e+08,4.437322e+08,20692167.0,20210602,2021-03-19 17:00:00,0.0,1.0,N,0.0,6875000.0,Active,20210602.0,C000014267,30.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Wiraswasta,3.0,695.0,0
4,99149981,1895121000014213,329701002968503,2021-05-18 17:00:00,2021-06-03 17:00:00,2021-05-18 17:00:00,0.0,0.0,1272266.0,1272266.0,0.0,3.417080e+05,889101.0,10000.0,10000,58.0,203.0,4.833262e+07,1.371915e+08,-88858929.0,20210603,2021-03-16 17:00:00,0.0,1.0,Y,1451371.0,3750000.0,Active,20210603.0,C000013937,30.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Wiraswasta,8.0,599.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8828,99167509,1895121000018923,441101019865539,2021-06-08 17:00:00,2021-06-24 17:00:00,2021-06-19 17:00:00,400881.0,8199.0,1167759.0,775077.0,392682.0,8.697219e+05,3442053.0,25000.0,98657,135.0,648.0,1.768544e+08,1.516753e+08,25179120.0,20210622,2021-03-26 17:00:00,13.0,1.0,Y,392682.0,1250000.0,Active,20210622.0,C000018643,26.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Karyawan,9.0,494.0,13
8829,99167506,1895121000018920,31901036550501,2021-06-08 17:00:00,2021-06-24 17:00:00,2021-06-20 17:00:00,390134.0,390134.0,1370402.0,1370402.0,0.0,7.315364e+05,2292727.0,95999.0,601694,513.0,864.0,5.302854e+08,5.501096e+08,-19824156.0,20210622,2021-03-26 17:00:00,0.0,1.0,N,0.0,6875000.0,Active,20210622.0,C000018640,37.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Wiraswasta,3.0,676.0,0
8830,99167505,1895121000018919,335201048406531,2021-06-08 17:00:00,2021-06-24 17:00:00,2021-06-19 17:00:00,601380.0,26874.0,1989140.0,1414634.0,574506.0,5.415993e+05,2136224.0,25000.0,185594,891.0,756.0,4.823319e+08,5.146573e+08,-32325372.0,20210622,2021-03-26 17:00:00,13.0,1.0,Y,574506.0,3750000.0,Active,20210622.0,C000018639,24.0,Perempuan,Menikah,Sekolah Menengah Atas (SMA),Karyawan,8.0,598.0,13
8831,99167535,1895121000018949,176101001510504,2021-06-08 17:00:00,2021-06-24 17:00:00,2021-06-20 17:00:00,237956.0,26000.0,2468156.0,2256200.0,211956.0,9.724074e+04,107000.0,91500.0,91500,27.0,108.0,4.050000e+06,3.766500e+06,283500.0,20210622,2021-03-26 17:00:00,13.0,1.0,Y,211956.0,2500000.0,Active,20210622.0,C00001866

In [141]:
#Labeling NPL & PL per ODA
label_juni = []
for i in range(dfinal_juni.shape[0]):
    if dfinal_juni.dpd_x.iloc[i] >= 30:
        label_juni.append(1) #NPL
    else :
        label_juni.append(0) #PL
        
dfinal_juni['label'] = label_juni

In [142]:
# Drop dpd column and save it
dfinal_juni=dfinal_juni.drop(columns=['dpd_x','dpd_y'])
dfinal_juni.to_csv('data_billing_juni_new_dpd.csv')

# Start from here

In [40]:
bill_april = pd.read_csv('data_billing_april_new_dpd.csv')
bill_mei = pd.read_csv('data_billing_mei_new_dpd.csv')
bill_juni = pd.read_csv('data_billing_juni_new_dpd.csv')

In [41]:
bill_april

,Unnamed: 0,acid,oda,acctno,bill_gen_date,payment_due_date,payment_date,bill_amt,adj_amt,total_bill,total_adj,total_adj_bill,avg_saldol1m,max_saldol1m,min_saldol1m,median_saldol1m,freq_creditl1m,freq_debitl1m,credit_amtl1m,debit_amtl1m,disp_income_amtl1m,ds_ts,acct_opn_date,collectibility,hold_flag,hold,limit,oda_status,posisi,cif_id,age,detail_gender,detail_marital_status,detail_last_education,detail_occupation_type,grade,score,label
0,0,99147351,1895121000014173,202201005078507,2021-03-16 17:00:00,2021-04-01 17:00:00,2021-03-28 17:00:00,19610751.0,19610751.0,19610751.0,19610751.0,0.0,1.863760e+07,92261029.00,619588.00,13592398,216.0,1431.0,2.717562e+09,2.950181e+09,-232619094.0,20210401,2021-03-09 17:00:00,1.0,N,0.0,20000000.0,Active,20210401.0,C000013897,32.0,Perempuan,Menikah,Strata 2 (S2),Lain-Lain,3.0,761.0,0
1,1,99143902,1895121000014132,617401028725530,2021-03-16 17:00:00,2021-04-01 17:00:00,2021-03-28 17:00:00,652134.0,652134.0,652134.0,652134.0,0.0,1.777416e+07,20440727.40,2440727.40,20440728,27.0,108.0,9.242100e+04,4.862345e+08,-486142074.0,20210401,2021-03-02 17:00:00,1.0,N,0.0,20000000.0,Active,20210401.0,C000013856,37.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Lain-Lain,7.0,608.0,0
2,2,99144562,1895121000014143,15801038925500,2021-03-16 17:00:00,2021-04-01 17:00:00,2021-03-28 17:00:00,350623.0,350623.0,350623.0,350623.0,0.0,9.073346e+05,5463976.35,115976.35,364477,459.0,594.0,5.413664e+08,5.181705e+08,23195943.0,20210401,2021-03-03 17:00:00,1.0,N,0.0,20000000.0,Active,20210401.0,C000013867,32.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Lain-Lain,7.0,563.0,0
3,3,99150458,1895121000014325,701401009164538,2021-03-17 17:00:00,2021-04-02 17:00:00,2021-04-05 17:00:00,2770427.0,2770427.0,2770427.0,2770427.0,0.0,1.122130e+05,2665625.00,13125.00,13650,54.0,162.0,1.208486e+08,1.208628e+08,-14175.0,20210402,2021-03-17 17:00:00,1.0,Y,2770427.0,6875000.0,Active,20210402.0,C000014066,32.0,Laki-laki,Menikah,Sekolah Menengah Atas (SMA),Karyawan,3.0,700.0,0
4,4,99150393,1895121000014277,207401009663501,2021-03-17 17:00:00,2021-04-02 17:00:00,2021-03-29 17:00:00,676504.0,676504.0,676504.0,676504.0,0.0,2.295570e+05,1658352.00,38952.00,58601,486.0,1134.0,3.719190e+08,3.793149e+08,-7395894.0,20210402,2021-03-16 17:00:00,1.0,N,0.0,1250000.0,Active,20210402.0,C000014012,21.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Lain-Lain,9.0,490.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9217,9217,99150035,1895121000014267,409901021724537,2021-04-14 17:00:00,2021-04-30 17:00:00,2021-05-14 17:00:00,1851065.0,0.0,1851065.0,0.0,1851065.0,2.226880e+05,1763662.00,50125.00,55625,250.0,725.0,1.085027e+08,1.364884e+08,-27985700.0,20210430,2021-03-16 17:00:00,1.0,Y,1851065.0,2500000.0,Active,20210430.0,C000013998,24.0,Laki-laki,Lajang,Sekolah Menengah Atas (SMA),Karyawan,9.0,567.0,0
9218,9218,99149971,1895121000014204,420301000164505,2021-04-14 17:00:00,2021-04-30 17:00:00,2021-04-25 17:00:00,3009198.0,3009198.0,3009198.0,3009198.0,0.0,2.969714e+06,13238704.00,507373.00,2596873,375.0,1100.0,7.983884e+08,8.343600e+08,-35971625.0,20210430,2021-03-16 17:00:00,1.0,N,0.0,17000000.0,Active,20210430.0,C000013928,39.0,Laki-laki,Duda/Janda,Diploma 3 (D3),Profesional,8.0,442.0,0
9219,9219,99150953,1895121000014401,105301012665503,2021-04-14 17:00:00,2021-04-30 17:00:00,2021-04-25 17:00:00,1592920.0,1592920.0,1592920.0,1592920.0,0.0,4.382200e+04,43822.00,43822.00,43822,175.0,225.0,3.050000e+08,3.050000e+08,0.0,20210430,2021-03-18 17:00:00,1.0,N,0.0,6875000.0,Active,20210430.0,C000014125,37.0,Laki-laki,Menikah,Sekolah Menengah Pertama (SMP),Lain-Lain,3.0,691.0,0
9220,9220,99157417,1895121000016694,5101105404502,2021-04-14 17:00:00,2021-04-30 17:00:00,2021-04-26 17:00:00,1558995.0,1558995.0,1558995.0,1558995.0,0.0,9.945178e+05,8081747.80,54162.80,342207,375.0,1625.0,5.115991e+08,4.604336e+08,51165500.0,20210430,2021-03-22 17:00:00,1.0,Y,51000

In [2]:
'''
# %%
## SPLIT DATASET INTO DATA TRAIN AND DATA TEST
data_train, data_test = train_test_split(Dataset, test_size=0.30, random_state=24)

data_train.to_csv("%s_data_train_agg.csv" % str(date.today().strftime("%Y%m%d")), index=False)
data_test.to_csv("%s_data_test_agg.csv" % str(date.today().strftime("%Y%m%d")), index=False)
'''

'\n# %%\n## SPLIT DATASET INTO DATA TRAIN AND DATA TEST\ndata_train, data_test = train_test_split(Dataset, test_size=0.30, random_state=24)\n\ndata_train.to_csv("%s_data_train_agg.csv" % str(date.today().strftime("%Y%m%d")), index=False)\ndata_test.to_csv("%s_data_test_agg.csv" % str(date.today().strftime("%Y%m%d")), index=False)\n'

In [25]:
#x=pd.read_csv('20210222_dataset_all_trx_labeled.csv')

In [27]:
# %%
## SPLIT DATASET INTO DATA TRAIN AND DATA TEST
#data_train, data_test = train_test_split(x, test_size=0.30, random_state=24)

#data_train.to_csv("%s_data_train.csv" % str(date.today().strftime("%Y%m%d")), index=False)
#data_test.to_csv("%s_data_test.csv" % str(date.today().strftime("%Y%m%d")), index=False)

x.head()

,cif_id,plafond,purchase_amt,loan_term,total_spent_l3d,cumsum_bill_adj,avg_credit,pct_of_credit,rule_1_all,rule_2_all,rule_3_all,comb_rule_all,pct_of_plf_pch,pct_of_plf_l3d,comb_rule_and,cumul_comb_rule_all,cumul_comb_rule_and,cumul_avg_rule_all,cumul_avg_rule_and,label
0,C000000004,5000000,625539,3,625539,217395.6538,1.051571e+08,0.002067,0,0,0,0,0.125108,0.125108,0,0,0,0.0,0.0,0
1,C000000004,5000000,3073014,6,3073014,555805.7988,8.937386e+07,0.006219,1,0,0,1,0.614603,0.614603,1,1,1,0.5,0.5,0
2,C000000001,5000000,524755,3,524755,182369.8543,3.263317e+07,0.005588,0,0,0,0,0.104951,0.104951,0,0,0,0.0,0.0,0
3,C000000001,5000000,699335,3,699335,243042.2237,3.406036e+07,0.007136,0,0,0,0,0.139867,0.139867,0,0,0,0.0,0.0,0
4,C000000001,5000000,608000,3,608000,454342.4903,3.235873e+07,0.014041,0,0,0,0,0.121600,0.121600,0,0,0,0.0,0.0,0


# Split Dataset dalam grup CIF_ID yang sama

In [38]:
'''
train_inds, test_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7).split(x, groups=x['cif_id']))

train = x.iloc[train_inds]
test = x.iloc[test_inds]

train.to_csv("%s_data_train_all_trx_group.csv" % str(date.today().strftime("%Y%m%d")), index=False)
test.to_csv("%s_data_test_all_trx_group.csv" % str(date.today().strftime("%Y%m%d")), index=False)
'''

In [37]:
train.cif_id.nunique()

8117

In [36]:
test.cif_id.nunique()

2030

In [35]:
x.cif_id.nunique()

10147